<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Basin-scale simulation

This notebook goes beyond the basic example of the [first notebook](1_basic-example.ipynb) to show how to simulate sedimentation in an entire basin using StratigraPy.

### Imports

Let's first import all the required packages and components:

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, median_filter
from scipy.interpolate import PchipInterpolator
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmocean
import colorsys
import pyvista as pv
import vtk

from landlab.components import FlowDirectorD8, FlowAccumulator

from stratigrapy import RasterModelGrid
from stratigrapy.components import SeaLevelCalculator, SourceProducer, WaterDrivenRouter
from stratigrapy.plot import extract_tie_centered_layers

### Functions

And define some utility functions:

In [ ]:
def add_text(plotter, text, position, font_size, font_file, color, opacity=1.,
             orientation=0., offset=0., justification_horizontal=None,
             justification_vertical=None):
    """
    Add a text to a PyVista plotter with the option to control its orientation.
    """
    # Create text actor
    text_actor = vtk.vtkTextActor()
    text_actor.SetInput(text)

    # Set text properties
    text_prop = text_actor.GetTextProperty()
    text_prop.SetFontFamily(4)
    text_prop.SetFontFile(font_file)
    text_prop.SetFontSize(font_size)
    text_prop.SetColor(*color)
    text_prop.SetOpacity(opacity)
    if justification_horizontal == 'right':
        text_prop.SetJustificationToRight()
    else:
        text_prop.SetJustificationToCentered()
    if justification_vertical == 'center':
        text_prop.SetVerticalJustificationToCentered()
    elif justification_vertical == 'top':
        text_prop.SetVerticalJustificationToTop()
    text_prop.SetLineOffset(offset)

    # Set position
    if len(position) == 3:
        coordinate = vtk.vtkCoordinate()
        coordinate.SetCoordinateSystemToWorld()
        coordinate.SetValue(*position)
        position = coordinate.GetComputedDisplayValue(plotter.renderer)
    text_actor.SetPosition(position[0], position[1])
    # Set rotation
    text_actor.SetOrientation(orientation)

    # Add to renderer
    plotter.renderer.AddActor2D(text_actor)

    return text_actor

In [ ]:
def _build_sand_cmap(light_fraction_1,
                     light_fraction_2,
                     light_fraction_3,
                     light_fraction_4,
                     use_gold_sand=False,
                     reverse=False,
                     name='sand'):
    """
    Builds a colormap following a sandy color scheme.

    Parameters
    ----------
    light_fraction_1 : float
        Light fraction of the first color.
    light_fraction_2 : float
        Light fraction of the second color.
    light_fraction_3 : float
        Light fraction of the third color.
    light_fraction_4 : float
        Light fraction of the fourth color.
    use_gold_sand : bool (default False)
        If true, uses Gold Sand as third color, otherwise uses Yuma Sand.
    reverse : bool (default False)
        If true, reverses the color list.
    name : str (default 'sand')
        Name of the colormap.

    Returns
    -------
    cmap
        The colormap.
    """
    mississippi_mud = (15/360, 0.29 + light_fraction_1*0.29, 0.14)
    rio_grande_mud = (22/360, 0.48 + light_fraction_2*0.48, 0.33)
    yuma_sand = (50/360, 0.89 + light_fraction_3*0.89, 0.78)
    drifted_sand = (55/360, 0.94 + light_fraction_4*0.94, 0.40)
    color_list = [colorsys.hls_to_rgb(*mississippi_mud) + (1,),
                  colorsys.hls_to_rgb(*rio_grande_mud) + (1,),
                  colorsys.hls_to_rgb(*yuma_sand) + (1,),
                  colorsys.hls_to_rgb(*drifted_sand) + (1,)]
    if use_gold_sand:
        gold_sand = (46/360, 0.82 + light_fraction_3*0.82, 0.83)
        color_list[2] = colorsys.hls_to_rgb(*gold_sand) + (1,)
    if reverse:
        color_list = color_list[::-1]

    return mcolors.LinearSegmentedColormap.from_list(name, color_list)

### Setting

Now let's define a custom colormap using one of the utility functions for the final animation:

In [ ]:
sand_light = _build_sand_cmap(0.25, 0.08543330133998818, -0.4679754966612923, 0.0,
                              use_gold_sand=False, name='sand_light')

## 1. Setup and simulation

We'll simulate over a much longer time than in the previous examples &ndash; 8 million years &ndash; but still with time steps of a hundred years:

In [ ]:
timestep = 100.
runtime = 8000000.
n_iterations = int(runtime/timestep)

To keep the computational burden reasonable, we create a grid with a similar number of cells, and just use larger cells to define a larger simulation domain:

In [ ]:
grid = RasterModelGrid((25, 29),
                       xy_spacing=(5000., 5000.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//1000 + 100,
                       number_of_layers_to_fuse=1000,
                       number_of_top_layers=100)

Here we'll keep all the boundaries closed to fill up the domain more quickly:

In [ ]:
grid.set_closed_boundaries_at_grid_edges(True, True, True, True)

Finally, we create a bowl-like initial topography, this time adding some random features:

In [ ]:
rng = np.random.default_rng(101010)

elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 10.
elevation[(grid.x_of_node > 30000.) & (grid.x_of_node < 110000.) & (grid.y_of_node > 40000.) & (grid.y_of_node < 80000.)] = -60.
for scale, sigma in zip([15., 1.5], [10., 1.]):
    elevation += rng.normal(scale=scale, size=grid.number_of_nodes)
    elevation[:] = gaussian_filter(elevation.reshape(grid.shape), sigma).ravel()
elevation[:] *= 30.
elevation[:] = median_filter(elevation.reshape(grid.shape), 3).ravel()

In [ ]:
grid.imshow('topographic__elevation', var_name='Elevation', var_units='m',
            grid_units=['m', 'm'], cmap='gist_earth')

The subsidence follows a similar bowl-like shape, so that sediments mainly accumulate in the center of the domain:

In [ ]:
rng = np.random.default_rng(42)

uplift = grid.add_zeros('uplift__rate', at='node', clobber=True)
uplift[(grid.x_of_node > 20000.) & (grid.x_of_node < 120000.) & (grid.y_of_node > 20000.) & (grid.y_of_node < 100000.)] -= 5e-5
uplift[(grid.x_of_node > 40000.) & (grid.x_of_node < 100000.) & (grid.y_of_node > 40000.) & (grid.y_of_node < 80000.)] -= 5e-5
for scale, sigma in zip([5e-5, 5*7e-7], [7., 1.]):
    uplift += rng.normal(scale=scale, size=grid.number_of_nodes)
    uplift[:] = gaussian_filter(uplift.reshape(grid.shape), sigma).ravel()

In [ ]:
grid.imshow('uplift__rate', var_name='Uplift rate', var_units='m/yr',
            grid_units=['m', 'm'], cmap='magma_r')

The sources of water and sediments will be more complex than in the previous examples, with three sources, each comprising multiple cells and active at different times. Here we need to define the location of the sources, and the control points of water and sediment influx:

In [ ]:
source_1_xy = [
    [5000., 115000.], [10000., 115000.], [15000., 115000.], [20000., 115000.], [25000., 115000.], [30000., 115000.], [35000., 115000.], [40000., 115000.],
    [5000., 80000.], [5000., 85000.], [5000., 90000.], [5000., 95000.], [5000., 100000.], [5000., 105000.], [5000., 110000.]
]
source_time = [0., 500000., 3000000., 7000000., 8000000.] # yr
water_source_1_influx = [300., 315., 200., 260., 240.] # m/yr
sediment_source_1_influx = [[14000., 6000.], [14000., 7000.], [6000., 2000.], [13000., 2500.], [9000., 500.]] # m3/yr

source_2_xy =[
    [5000., 5000.], [5000., 10000.], [5000., 15000.], [5000., 20000.], [5000., 25000.], [5000., 30000.], [5000., 35000.], [5000., 40000.],
    [10000., 5000.], [15000., 5000.], [20000., 5000.]
]
water_source_2_influx = [360., 300., 100., 0., 0.] # m/yr
sediment_source_2_influx = [[12000., 8000.], [7000., 3000.], [500., 1000.], [0., 0.], [0., 0.]] # m3/yr

source_3_xy = [
    [135000., 5000.], [135000., 10000.], [135000., 15000.], [135000., 20000.], [135000., 25000.], [135000., 30000.], [135000., 35000.], [135000., 40000.],
    [135000., 45000.], [135000., 50000.], [135000., 55000.], [135000., 60000.], [135000., 65000.], [135000., 70000.], [135000., 75000.], [135000., 80000.],
    [135000., 85000.], [135000., 90000.], [135000., 95000.], [135000., 100000.], [135000., 105000.], [135000., 110000.], [135000., 115000.],
    [100000., 115000.], [105000., 115000.], [110000., 115000.], [115000., 115000.], [120000., 115000.], [125000., 115000.], [130000., 115000.],
    [120000., 5000.], [125000., 5000.], [130000., 5000.],
]
water_source_3_influx = [0., 0., 40., 340., 280.] # m/yr
sediment_source_3_influx = [[0., 0.], [0., 0.], [500., 50.], [8000., 2000.], [10000., 6000.]] # m3/yr

At each iteration, water and sediment influxes are interpolated based on those control points, here using a monotonic spline interpolation. Let's visualize how it will look like:

In [ ]:
fig = plt.figure(layout='constrained', figsize=(12, 6))
subfigs = fig.subfigures(nrows=1, ncols=2, width_ratios=[1, 0.4])

axs = subfigs[0].subplots(nrows=2, ncols=3, sharex=True, sharey='row')
x = np.linspace(0., 8000000., 1000)
for j, source in enumerate([
    (water_source_1_influx, sediment_source_1_influx, 'tab:blue'),
    (water_source_2_influx, sediment_source_2_influx, 'tab:orange'),
    (water_source_3_influx, sediment_source_3_influx, 'tab:green')
]):
    interpolator = PchipInterpolator(source_time, source[0])
    axs[0, j].plot(x, interpolator(x), c=source[2])
    axs[0, j].set_title('Source ' + str(j + 1), c=source[2], fontweight='bold')
    sediment_influx = np.asarray(source[1])
    for i, (label, color) in enumerate(zip(['First class', 'Second class'], ['#cd34b5', '#ffd700'])):
        interpolator = PchipInterpolator(source_time, sediment_influx[:, i])
        axs[1, j].plot(x, interpolator(x), c=color, label=label)
    axs[1, j].set(xlabel='Time (yr)')
axs[1, 1].legend()
axs[0, 0].set(ylabel='Water influx (m/yr)')
axs[1, 0].set(ylabel=r'Sediment influx (m$^3$/yr)')
subfigs[0].align_ylabels()

axs = subfigs[1].subplots(nrows=2, ncols=1)
for j, xy in enumerate([source_1_xy, source_2_xy, source_3_xy]):
    xy = np.asarray(xy)
    axs[0].scatter(xy[:, 0], xy[:, 1])
axs[0].set(xlabel='x (m)', ylabel='y (m)', aspect='equal',
           xlim=(grid.x_of_node.min(), grid.x_of_node.max()),
           ylim=(grid.y_of_node.min(), grid.y_of_node.max()))
axs[1].set_axis_off();

And add them to a `SourceProducer` component, which will create the water and sediment influx maps for each iteration:

In [ ]:
source_xy = source_1_xy + source_2_xy + source_3_xy
water_source_influx = len(source_1_xy)*[water_source_1_influx] + len(source_2_xy)*[water_source_2_influx] + len(source_3_xy)*[water_source_3_influx]
sediment_source_influx = len(source_1_xy)*[sediment_source_1_influx] + len(source_2_xy)*[sediment_source_2_influx] + len(source_3_xy)*[sediment_source_3_influx]

In [ ]:
sp = SourceProducer(grid,
                    source_xy=source_xy,
                    water_source_time=source_time,
                    water_source_influx=water_source_influx,
                    sediment_source_time=source_time,
                    sediment_source_influx=sediment_source_influx,
                    interpolation='monotonic')

Let's follow a similar approach with sea level:

In [ ]:
control_time = [0., 500000., 1500000., 3000000., 5500000., 7000000., 8000000.] # yr
control_sea_level = [0., -5., -30., -10., -80., -50., -40.] # m

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
x = np.linspace(0., 8000000., 1000)
interpolator = PchipInterpolator(control_time, control_sea_level)
ax.plot(x, interpolator(x))
ax.set(xlabel='Time (m)', ylabel='Sea level (m)');

In [ ]:
slc = SeaLevelCalculator(grid,
                         control_time=control_time,
                         control_sea_level=control_sea_level,
                         interpolation='monotonic')

To reduce the computational burden, let's use single flow direction:

In [ ]:
fd = FlowDirectorD8(grid)

In [ ]:
fa = FlowAccumulator(grid, flow_director=fd)

And use a single component for sediment transport:

In [ ]:
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-7, 1e-8],
                        transportability_mar=[1e-9, 1e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-11,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track=['surface_water__discharge', 'bathymetric__depth'])

Here we track two fields: the discharge and the bathymetry. Note that discharge is computed everywhere, including in the marine domain, because components like `WaterDrivenRouter` still use it there, just with a decreasing magnitude as the bathymetry increases.

We can now run the simulation, which takes much more time than the previous examples:

In [ ]:
for i in tqdm(range(n_iterations)):
    elevation += uplift*timestep
    slc.run_one_step(timestep)
    sp.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean,
                             surface_water__discharge=np.mean,
                             bathymetric__depth=np.mean)
grid.stacked_layers.fuse(finalize=True,
                         time=np.mean,
                         surface_water__discharge=np.mean,
                         bathymetric__depth=np.mean)

And visualize the result:

In [ ]:
fig, ax = plt.subplots()

raster_x = grid.x_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)

pc = ax.pcolormesh(raster_x, raster_y, raster_z, cmap=cmocean.cm.topo,
                   norm=mcolors.CenteredNorm(grid.at_grid['sea_level__elevation']))
fig.colorbar(pc, ax=ax, label='Elevation (m)')

ax.set(xlabel='x (m)', ylabel='y (m)', aspect='equal');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'bathymetric__depth', i_x=10, cmap=cmocean.cm.deep, zorder=2)
fig.colorbar(pc[0], ax=ax, label='Water depth (m)')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 10]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 10]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3.5))

# Sediments
pc = grid.plot_layers(ax, 'composition', i_x=10, i_class=1, mask_wedges=True, cmap='pink', zorder=2)
fig.colorbar(pc[0], ax=ax, label='Fraction of the second sediment class')

raster_y = grid.y_of_node[grid.core_nodes].reshape(grid.cell_grid_shape)[:, 10]
raster_z = grid.at_node['topographic__elevation'][grid.core_nodes].reshape(grid.cell_grid_shape)[:, 10]
# Sea level
fill_sea = ax.fill_between(raster_y, raster_z, grid.at_grid['sea_level__elevation'],
                           color='#c6dbef', zorder=0)
# Bedrock
ymin, ymax = ax.get_ylim()
ax.fill_between(raster_y, raster_z, ymin, color='#d9d9d9', zorder=1)

ax.set(xlabel='y (m)', ylabel='z (m)', ylim=(ymin, ymax));

## 2. 3D animation

Let's run the same simulation again, but this time using PyVista to save an animation of the basin filling up through time.

First, we need to re-initialize the grid and components:

In [ ]:
# Define the grid
grid = RasterModelGrid((25, 29),
                       xy_spacing=(5000., 5000.),
                       number_of_classes=2,
                       initial_allocation=n_iterations//1000 + 100,
                       number_of_layers_to_fuse=1000,
                       number_of_top_layers=100)
grid.set_closed_boundaries_at_grid_edges(True, True, True, True)

# Define the initial topography
rng = np.random.default_rng(101010)
elevation = grid.add_zeros('topographic__elevation', at='node', clobber=True)
elevation += 10.
elevation[(grid.x_of_node > 30000.) & (grid.x_of_node < 110000.) & (grid.y_of_node > 40000.) & (grid.y_of_node < 80000.)] = -60.
for scale, sigma in zip([15., 1.5], [10., 1.]):
    elevation += rng.normal(scale=scale, size=grid.number_of_nodes)
    elevation[:] = gaussian_filter(elevation.reshape(grid.shape), sigma).ravel()
elevation[:] *= 30.
elevation[:] = median_filter(elevation.reshape(grid.shape), 3).ravel()

# Define the subsidence
rng = np.random.default_rng(42)
uplift = grid.add_zeros('uplift__rate', at='node', clobber=True)
uplift[(grid.x_of_node > 20000.) & (grid.x_of_node < 120000.) & (grid.y_of_node > 20000.) & (grid.y_of_node < 100000.)] -= 5e-5
uplift[(grid.x_of_node > 40000.) & (grid.x_of_node < 100000.) & (grid.y_of_node > 40000.) & (grid.y_of_node < 80000.)] -= 5e-5
for scale, sigma in zip([5e-5, 5*7e-7], [7., 1.]):
    uplift += rng.normal(scale=scale, size=grid.number_of_nodes)
    uplift[:] = gaussian_filter(uplift.reshape(grid.shape), sigma).ravel()

# Define the components for water and sediment sources, sea level variation, water flow, and sediment transport
sp = SourceProducer(grid,
                    source_xy=source_xy,
                    water_source_time=source_time,
                    water_source_influx=water_source_influx,
                    sediment_source_time=source_time,
                    sediment_source_influx=sediment_source_influx,
                    interpolation='monotonic')
slc = SeaLevelCalculator(grid,
                         control_time=control_time,
                         control_sea_level=control_sea_level,
                         interpolation='monotonic')
fd = FlowDirectorD8(grid)
fa = FlowAccumulator(grid, flow_director=fd)
wdr = WaterDrivenRouter(grid,
                        transportability_cont=[1e-7, 1e-8],
                        transportability_mar=[1e-9, 1e-10],
                        wave_base=15.,
                        max_erosion_rate_sed=1e-2,
                        max_erosion_rate_br=1e-11,
                        bedrock_composition=[0.7, 0.3],
                        fields_to_track=['surface_water__discharge', 'bathymetric__depth'])

Then we define the PyVista plotter and the different components that will be visible in the animation, like the bedrock and the different sediment sources:

In [ ]:
p = pv.Plotter(off_screen=True, window_size=(3840, 2160), border=True, lighting='three lights')

# Text setting
font_file = '/Users/grongier/Library/Fonts/SourceSansPro-Regular.ttf'
label_color = (0.35, 0.35, 0.35)
font_size = 54
title_font_size = 64

# Add the surface of the bedrock
xy = grid.xy_of_cell.reshape(*grid.cell_grid_shape, 2)
z = grid.at_node['topographic__elevation'].reshape(grid.shape)[1:-1, 1:-1]
bedrock_surface = pv.StructuredGrid(xy[..., 0], xy[..., 1], z)
bedrock_actor = p.add_mesh(bedrock_surface, color=(0.8, 0.8, 0.8), smooth_shading=True, ambient=0.1)

# Add the lines representing the locations of the sediment sources
z_sources = 70.
line_kwargs = {'color': [0., 0., 0.], 'width': 12, 'connected': True}
source_1_line = np.array([[5000., 80000., z_sources],
                          [5000., 115000., z_sources],
                          [40000., 115000., z_sources]])
source_1_actor = p.add_lines(source_1_line, **line_kwargs)
source_1_actor.GetProperty().SetOpacity(1.)
source_2_line = np.array([[5000., 40000., z_sources],
                          [5000., 5000., z_sources],
                          [20000., 5000., z_sources]])
source_2_actor = p.add_lines(source_2_line, **line_kwargs)
source_2_actor.GetProperty().SetOpacity(1.)
source_3_line = np.array([[120000., 5000., z_sources],
                          [135000., 5000., z_sources],
                          [135000., 115000., z_sources],
                          [100000., 115000., z_sources]])
source_3_actor = p.add_lines(source_3_line, **line_kwargs)
source_3_actor.GetProperty().SetOpacity(1.)

# Add the lines representing the scale
origin = [-27000., 45000., 0.]
scale = [10000., -10000., 100.]
scale_line = np.array([origin, [origin[0] + scale[0], origin[1], origin[2]],
                       origin, [origin[0], origin[1] + scale[1], origin[2]],
                       origin, [origin[0], origin[1], origin[2] + scale[2]]])
p.add_lines(scale_line, color=label_color, width=6, connected=False)

# Add some vertical exaggeration
p.set_scale(zscale=100)

# Update the camera's position
p.camera_position = [
     (-52000., -80000., 120000.),
     (70000., 60000., -21000.),
     (0., 0., 1.)
]

# Add the colorbar
cbar = p.add_scalar_bar(title='Fraction of sand', n_labels=3, title_font_size=title_font_size,
                        label_font_size=font_size, color=label_color, width=0.15,
                        height=0.05, position_x=0.78, position_y=0.07, fmt='%.g')
for prop in [cbar.GetTitleTextProperty(), cbar.GetLabelTextProperty()]:
    prop.SetFontFamily(4)
    prop.SetFontFile(font_file)
    prop.SetLineOffset(15.)
lookup_table = pv.LookupTable(cmap=sand_light)
lookup_table.SetRange(0., 1.)
cbar.SetLookupTable(lookup_table)

# Add the labels of the scale
add_text(p, '10 km', (origin[0] + scale[0]/2., origin[1], origin[2] + 200.), font_size,
         font_file, label_color, orientation=25.)
add_text(p, '10 km', (origin[0], origin[1] + scale[1]/2., origin[2]), font_size,
         font_file, label_color, orientation=-55., offset=12., justification_vertical='top')
add_text(p, '100 m', (origin[0] - 350., origin[1], origin[2] + 100.*scale[2]/2.),
         font_size, font_file, label_color, orientation=-71., justification_vertical='top')

# Add the labels of the sources
source_1_label = add_text(p, 'Sediment source 1', (22500., 115000., 100.*z_sources + 500.),
                          title_font_size, font_file, (0., 0., 0.), orientation=19., opacity=1.)
source_2_label = add_text(p, 'Sediment source 2', (5000., 22500., 100.*z_sources - 1200.), title_font_size,
                          font_file, (0., 0., 0.), orientation=-43., opacity=1., justification_vertical='top')
source_3_label = add_text(p, 'Sediment source 3', (135000., 60000., 100.*z_sources + 600.),
                          title_font_size, font_file, (0., 0., 0.), orientation=-23., opacity=1.)

# Add the time
time_actor = add_text(p, '0 year ', (3760, 2030), title_font_size, font_file, (0., 0., 0.),
                      justification_horizontal='right')

# Add some anti-aliasing
p.enable_anti_aliasing('ssaa')

Finally, let's run the simulation while saving the different frames making the animation, which adds a big overhead, so this cell takes much longer to run:

In [ ]:
# Start saving the animation
frame_step = 50
p.open_movie('stratigrapy_basin_animation.mp4', framerate=32)

# Prepare sediment source normalization
source_1_interpolator = PchipInterpolator(source_time, sediment_source_1_influx)
source_2_interpolator = PchipInterpolator(source_time, sediment_source_2_influx)
source_3_interpolator = PchipInterpolator(source_time, sediment_source_3_influx)
source_vmin = min(np.min(np.sum(sediment_source_1_influx, axis=1)),
                  np.min(np.sum(sediment_source_2_influx, axis=1)),
                  np.min(np.sum(sediment_source_3_influx, axis=1)))
source_vmax = max(np.max(np.sum(sediment_source_1_influx, axis=1)),
                  np.max(np.sum(sediment_source_2_influx, axis=1)),
                  np.max(np.sum(sediment_source_3_influx, axis=1)))

# Store the slices through the stratigraphy to speed things up
stratigraphy_slices = []

for i in tqdm(range(n_iterations + 1)):

    # Start by taking care of the animation
    if i%frame_step == 0:

        # Get the stratigraphy and property
        x, y, z, layers = extract_tie_centered_layers(grid, 'composition', i_class=1, axis=2, mask_wedges=True)

        # Update the surface of the bedrock
        p.remove_actor(bedrock_actor)
        bedrock_surface = pv.StructuredGrid(x[0], y[0], z[0])
        bedrock_actor = p.add_mesh(bedrock_surface, color=(0.8, 0.8, 0.8), smooth_shading=True, ambient=0.1)

        # Update the slices through the stratigraphy
        slice_kwargs = {'scalars': 'Fraction of sand', 'clim': [0., 1.], 'cmap': sand_light,
                        'show_scalar_bar': False, 'ambient': 0.2}
        for l in range(len(layers)):
            if l < len(stratigraphy_slices):
                for idx, i in enumerate([6, 20]):
                    stratigraphy_slices[l][0][idx].points = np.stack([x[l:l + 2, :, i].T, y[l:l + 2, :, i].T, z[l:l + 2, :, i].T], axis=-1).reshape(-1, 3)
                    stratigraphy_slices[l][0][idx]['Fraction of sand'] = np.tile(layers[l, :, i], (2, 1, 1)).T.ravel()
                for idx, j in enumerate([2, 8, 14, 20]):
                    stratigraphy_slices[l][1][idx].points = np.stack([x[l:l + 2, j].T, y[l:l + 2, j].T, z[l:l + 2, j].T], axis=-1).reshape(-1, 3)
                    stratigraphy_slices[l][1][idx]['Fraction of sand'] = np.tile(layers[l, j], (2, 1, 1)).T.ravel()
            else:
                slices_i = []
                for i in [6, 20]:
                    strat_slice = pv.StructuredGrid(x[l:l + 2, :, i], y[l:l + 2, :, i], z[l:l + 2, :, i])
                    strat_slice['Fraction of sand'] = np.tile(layers[l, :, i], (2, 1, 1)).T.ravel()
                    p.add_mesh(strat_slice, **slice_kwargs)
                    slices_i.append(strat_slice)
                slices_j = []
                for j in [2, 8, 14, 20]:
                    strat_slice = pv.StructuredGrid(x[l:l + 2, j], y[l:l + 2, j], z[l:l + 2, j])
                    strat_slice['Fraction of sand'] = np.tile(layers[l, j], (2, 1, 1)).T.ravel()
                    p.add_mesh(strat_slice, **slice_kwargs)
                    slices_j.append(strat_slice)
                stratigraphy_slices.append([slices_i, slices_j])

        # Update the sediment sources
        opacity = (np.sum(source_1_interpolator(wdr._time)) - source_vmin)/(source_vmax - source_vmin)
        source_1_label.GetTextProperty().SetOpacity(opacity)
        source_1_actor.GetProperty().SetOpacity(opacity)
        opacity = (np.sum(source_2_interpolator(wdr._time)) - source_vmin)/(source_vmax - source_vmin)
        source_2_label.GetTextProperty().SetOpacity(opacity)
        source_2_actor.GetProperty().SetOpacity(opacity)
        opacity = (np.sum(source_3_interpolator(wdr._time)) - source_vmin)/(source_vmax - source_vmin)
        source_3_label.GetTextProperty().SetOpacity(opacity)
        source_3_actor.GetProperty().SetOpacity(opacity)

        # Update time
        time_actor.SetInput('{:,}'.format(round(wdr._time/100000)*100000).replace(',', ' ') + ' years')

        # Save the frame
        p.write_frame()

    # Then run the model
    elevation += uplift*timestep
    slc.run_one_step(timestep)
    sp.run_one_step(timestep)
    fa.run_one_step()
    wdr.run_one_step(timestep)
    grid.stacked_layers.fuse(time=np.mean, surface_water__discharge=np.mean, bathymetric__depth=np.mean)

p.close()